In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import os

name ="13조 기능정의"

# "거래일자" date-time 으로 포맷팅
def convert_time_format(time_string):
    if '오후' in time_string:
        time_string = time_string.replace('오후','PM')
        
    elif '오전' in time_string:
        time_string = time_string.replace('오전','AM')
       
    return time_string

# 카테고리컬 데이터 시각화
def plot_categorical_distribution(dataframe):
    """
    데이터프레임의 각 범주형 데이터에 대한 분포를 시각화하는 함수
    
    :param dataframe: 범주형 데이터가 포함된 데이터프레임
    """
    # 범주형 데이터 추출
    categorical_cols = dataframe.select_dtypes(include=['object']).columns.tolist()

    # 범주형 데이터에 대한 countplot 그리기
    for col in categorical_cols:
        plt.figure(figsize=(10,6))
        sns.countplot(data=dataframe, x=col)
        plt.title(f'{col} Distribution', fontsize=15)
        plt.xticks(rotation=45, ha='right', fontsize=12)

# 카테고리컬 데이터 시각화 (파이차트)
def plot_piechart(df, column, figsize=(8,8), title=None, explode=None, autopct='%1.1f%%'):
    """
    Plot a pie chart for categorical data in a dataframe column.
    
    Args:
    - df: pandas dataframe
    - column: str, column name of categorical data
    - figsize: tuple, optional, figure size, default=(8,8)
    - title: str, optional, plot title
    - explode: list-like, optional, offsets of each wedge, default=None
    - autopct: str or callable, optional, format string or function to label the wedges with their numeric value, default='%1.1f%%'
    
    Returns:
    - None
    """
    #고유값 빈도수 체크
    counts = df[column].value_counts()
    
    # 플롯차트 생성
    plt.figure(figsize=figsize)
    plt.pie(counts, labels=counts.index, explode=explode, autopct=autopct)
    plt.axis('equal')
    if title:
        plt.title(title)
    plt.show()

# 두개의 컬럼을 선택하여 집계값을 내주는 함수    
def aggregate_columns(df, column1, column2):
    aggregated_data = df.groupby(column1)[column2].agg(['sum', 'mean', 'count','max','min'])
    median = df.groupby(column1)[column2].median()
    quantile_1 = df.groupby(column1)[column2].quantile(0.25)  # 1사분위수
    quantile_3 = df.groupby(column1)[column2].quantile(0.75)  # 3사분위수
    most_freq = df.groupby(column1)[column2].agg(lambda x: x.value_counts().index[0])
    uniq = df.groupby(column1)[column2].nunique()
    std = df.groupby(column1)[column2].std()
    var = df.groupby(column1)[column2].var()

    return aggregated_data, median, quantile_1,quantile_3, most_freq, uniq, std, var


# 정규표현식을 이용하여 원하는 형식으로 변환시키기
def replace_strings_with_regex_patterns(df, column_name, regex_patterns, replacement_patterns):
    modified_df = df.copy()  # 원본 데이터프레임을 복사하여 수정된 데이터프레임 생성
    for index, row in modified_df.iterrows():
        for pattern, replacement in zip(regex_patterns, replacement_patterns):
            regex = re.compile(pattern)
            match = regex.search(row[column_name])
            if match:
                replaced_string = re.sub(pattern, replacement, match.group(0))
                modified_df.at[index, column_name] = replaced_string
    return modified_df


# 구매횟수별 쿠폰사용분포 그리기(ex. 2번구매한 사람은 2번째에 구매할 때 사용한 쿠폰만)
def purchase_count_coupon_distribution(df, purchase_count):
    
    value_counts = df['고객id'].value_counts()
    filtered_df = df[df['고객id'].isin(value_counts[value_counts == purchase_count].index)]


    # 두 개의 열을 기준으로 오름차순과 내림차순으로 정렬
    sorted_df = filtered_df.sort_values(['고객id', '거래일자'], ascending=[True, True])  
    if purchase_count == 1:
        new_df = sorted_df
    else :
        new_df = sorted_df.iloc[1::purchase_count]
    
    # 데이터프레임에서 카운트 플롯을 생성할 열을 지정합니다.
    column_to_plot = '쿠폰이름'  


    # 열의 값을 기준으로 빈도수를 계산하고 정렬합니다.
    value_counts2 = new_df[column_to_plot].value_counts().sort_values(ascending=False)


    # 그래프에 설정
    plt.figure(figsize=(15, 9))

    # 카운트 플롯 생성
    sns.countplot(data=new_df, x=column_to_plot, order=value_counts2.index)


    # 그래프에 설정
    plt.title(f"구매횟수별({purchase_count}번째) 쿠폰사용분포")


    # 그래프 출력
    plt.show()

# 구매횟수별 카테고리 빈도 시각화
def purchase_count_by_category(df):
    value_counts = df[df['유형']=='PAYMENT']['고객id'].value_counts()
    one_purchase = value_counts[value_counts == 1].index
    two_purchase = value_counts[value_counts == 2].index
    three_purchase = value_counts[value_counts == 3].index
    four_purchase = value_counts[value_counts == 4].index
    five_more_purchase = value_counts[value_counts >= 5].index

    one_data = df[df['고객id'].isin(one_purchase)]
    two_data = df[df['고객id'].isin(two_purchase)]
    three_data = df[df['고객id'].isin(three_purchase)]
    four_data = df[df['고객id'].isin(four_purchase)]
    five_more_data = df[df['고객id'].isin(five_more_purchase)]

    # subplot 생성
    fig, ax = plt.subplots(figsize=(12, 6))

    # 각 데이터에 대한 카테고리별 빈도 그래프 그리기
    category_counts_one = one_data['카테고리'].value_counts()
    category_counts_two = two_data['카테고리'].value_counts()
    category_counts_three = three_data['카테고리'].value_counts()
    category_counts_four = four_data['카테고리'].value_counts()
    category_counts_five = five_more_data['카테고리'].value_counts()

    ax.bar(category_counts_one.index, category_counts_one.values, label='1번 구매한 고객')
    ax.bar(category_counts_two.index, category_counts_two.values, label='2번 구매한 고객')
    ax.bar(category_counts_three.index, category_counts_three.values, label='3번 구매한 고객')
    ax.bar(category_counts_four.index, category_counts_four.values, label='4번 구매한 고객')
    ax.bar(category_counts_five.index, category_counts_five.values, label='5번 구매한 고객')

    # 그래프 제목과 레이블 설정
    ax.set_xlabel('카테고리')
    ax.set_ylabel('빈도')
    ax.set_title('구매횟수별 카테고리 빈도')
    ax.tick_params(axis='x', rotation=45)

    # 범례 추가
    ax.legend()

    # 그래프 표시
    plt.tight_layout()
    plt.show()

   
    return one_data, two_data, three_data, four_data, five_more_data

# 구매횟수별 고객수
def purchase_count_by_customer(df, one_data, two_data, three_data, four_data, five_more_data):
    categories = ['1번 구매', '2번 구매', '3번 구매', '4번 구매', '5번이상 구매']
    customer_counts = [
        one_data['고객id'].nunique(),
        two_data['고객id'].nunique(),
        three_data['고객id'].nunique(),
        four_data['고객id'].nunique(),
        five_more_data['고객id'].nunique(),
    ]

    # 색상 설정
    colors = ['skyblue', 'lightblue', 'lightblue', 'lightblue', 'lightblue', 'lightblue']

    # 그래프 그리기
    fig, ax = plt.subplots()
    bars = ax.bar(categories, customer_counts, color=colors)

    # 막대 위에 수치 표시
    for bar in bars:
        yval = bar.get_height()
        ax.text(bar.get_x() + bar.get_width() / 2, yval, yval, ha='center', va='bottom')


    # 그래프 스타일 설정
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.tick_params(axis='x', bottom=False)
    ax.set_ylabel('고객 수')
    ax.set_title('구매횟수별 고객 수')

    plt.show()

def average_purchase_cycle(series)->float:
    """
    유니크한 고객 재구매 주기 평균계산
    """
    purchase_date=[]
    purchase_cycle=[]
    for _date in series:
        if len(series)==1:
            return 0
        else:
            purchase_date.append(_date)
            if len(purchase_date) == len(series):
                for i in range(len(purchase_date)):
                    if i == len(purchase_date)-1:
                        return sum(purchase_cycle) / len(purchase_cycle)
                    else:
                        purchase_cycle.append((purchase_date[i+1]-purchase_date[i]).days)


# 구매주기에 대한 시각화 함수
def show_repurchase_cycle(df)->None:
    id_counts =df["고객id"].value_counts()
    id_counts
    # 10 이상인 고객 ID를 선택합니다.
    selected_ids1 = id_counts[id_counts == 2].index.tolist()
    selected_ids2 = id_counts[id_counts == 3].index.tolist()
    selected_ids3 = id_counts[id_counts == 4].index.tolist()
    selected_ids4 = id_counts[id_counts == 5].index.tolist()
    def frequency_payment(customer_id):
        if customer_id in selected_ids1:
            return 2
        elif customer_id in selected_ids2:
            return 3
        elif customer_id in selected_ids3:
            return 4
        elif customer_id in selected_ids4:
            return 5
        else:
            return 6
        
    df['frequency_customer'] = df['고객id'].apply(frequency_payment)

    # 거래일자 컬럼으로 오름차순 정렬
    df_sorted_asc = df[['고객id','거래일자','frequency_customer']].sort_values('거래일자')
    #고객 id 별 재구매 주기 평균을 구하기
    series_repurchase_cycle=df_sorted_asc.groupby('고객id')['거래일자'].apply(average_purchase_cycle)
    repurchase_cycle=series_repurchase_cycle.to_frame()
    merged_df = pd.merge(repurchase_cycle, df_sorted_asc[['고객id','frequency_customer']], on='고객id')
    #시각화
    sns.boxplot(x=merged_df['frequency_customer'],y=merged_df['거래일자'],data=merged_df)
    plt.xlabel("구매 횟수")
    plt.ylabel("구매 일자(일)")
    plt.show()

#쿠폰의미 파악
def coupon_meaning(text):
    # 추출패턴 리스트
    pattern_list=[
        ['프로모션'],
        ['WELCOME'],
        ['증정'],
        ['VIP'],
        ['크리스마스'], #=>특정일 할인 : 특정 날짜에만 적용되는 쿠폰으로
        ['자유이용권'],
        ['팔로워'],
        ['재발급'],
        ['럭키룰렛'],
        ['네오아카데미','서강대학교 대학원 특별할인','B2B','Internship coupon','인턴'],
        ['신규'],
        ['마스터'],
        ['시크릿'],
        ['한정'],
        ['온라인'],
        ['런칭'],
        ['전용'],
        ["%쿠폰","%할인권","원 쿠폰","원 할인","원할인","% 할인쿠폰","% 할인 쿠폰","% 즉시할인","원 즉시할인","% 할인","%할인","즉시할인","할인 쿠폰","강의 쿠폰","보상 쿠폰","할인쿠폰","오픈지연 쿠폰","할인 COUPON"],
        ['무료','100% 쿠폰','100% 페이백'],
        ['분할결제','분할 결제'],
        ['재결제']
        ]
    # 카테고리화 리스트
    return_list=['프로모션','WELCOME','증정','VIP','특정일 할인','자유이용권','팔로워','재발급','럭키룰렛','기업체','신규','마스터',
                '시크릿','한정','온라인','런칭','전용','할인','무료','분할결제','재결제']
    # 패턴을 통한 카테고리화
    for i,pattern in enumerate(pattern_list):
        match_pattern=re.compile(r"|".join(pattern))
        matches = match_pattern.findall(text)
        if matches:
            return return_list[i]
        if i ==len(pattern_list)-1:
            return '-'

def change_coupon_mean_df(df):
    mean_df = df.copy()
    mean_df['쿠폰이름'] = df['쿠폰이름'].apply(coupon_meaning)
    return mean_df

def change_coupon_category_df(df):
    category_df = df.copy()
    category_df['쿠폰이름'] = category_df[['쿠폰이름','코스(상품) 이름','카테고리']].apply(coupon_category, axis=1)
    return category_df

# 쿠폰 카테고리 파악
def coupon_category(data):
    # 패턴추출 리스트
    pattern_list=[
    ['업무','생산성'],
    ['부동산','금융'],
    ['영상','3D','편집녀'],
    ['프로그래밍','한 번에 끝내는 AWS 기반 아키텍처 설계와 DevOps 초격차 패키지','DevOps 초격차 오픈 지연 보상','10개 기술스택으로 완성하는 풀스택 서버리스 프로젝트 with React','블록체인'],
    ['마케팅'],
    ['일러스트'],
    ['디자인'],
    ['데이터','사이언스'],
    ['부업','창업'],
    ['투자','재테크'],
    ]
    li_course=data['코스(상품) 이름'].split(':')
    if len(li_course)==1:
        check=data['쿠폰이름'].find(li_course[0])
    else:
        check=data['쿠폰이름'].find(li_course[1])
    if check == -1:
        pass
    else :
        return data['카테고리']
    # 패턴을 통한 카테고리화
    for i,pattern in enumerate(pattern_list):
        match_pattern=re.compile(r"|".join(pattern))
        matches = match_pattern.findall(data['쿠폰이름'])
        if matches:
            return data['카테고리']
        if i ==len(pattern_list)-1:
            if data['쿠폰이름']=='-':
                return '-'
            else:
                return 'ALL'

